### Análisis de Datos y Clasificación ABC en una Cadena de Suministros

Realizaremos un análisis exhaustivo de datos ficticios correspondientes a una cadena de suministros. Este proceso incluirá la visualización de datos para identificar patrones y tendencias, así como la clasificación ABC tanto de productos como de clientes. La visualización facilitará la comprensión de la distribución y el rendimiento, mientras que la clasificación ABC ayudará a categorizar y gestionar los elementos clave en la cadena de suministros.



In [67]:
import pandas as pd
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [68]:
#Lectura de los datos "Sales test"
df = pd.read_csv('sales.csv')
df.head()

,ORDER_NO,DATE,LINE,CUSTOMER_NO,ITEM,NS_ORDER,NS_SHIP
0,528758,"Tuesday, January 3, 2017",1,1358538.0,111931,70.0,70.0
1,528791,"Tuesday, January 3, 2017",1,1254798.0,1029071,10.0,10.0
2,528791,"Tuesday, January 3, 2017",2,1254798.0,1033341,10.0,10.0
3,528791,"Tuesday, January 3, 2017",3,1254798.0,1040827,5.0,5.0
4,528791,"Tuesday, January 3, 2017",4,1254798.0,10106111,10.0,10.0


* ORDER_NO: Identificador de la orden
* DATE: Fecha de la orden
* LINE: Línea de embarque
* CUSTOMER_NO: Identificador del cliente
* ITEM: Identificador del SKU
* NS_ORDER: Cantidad de items ordenados
* NS_SHIPED: Cantidad de items despachados

In [69]:
#Convierto la fecha de formato string a formato date
def string_to_date(date_str):
     return datetime.strptime(date_str, "%A, %B %d, %Y")

df['DATE'] = df['DATE'].apply(string_to_date)

### Volumen diario de ordenes

In [62]:
#Volumen diario
df_daily_volume = df[['DATE','NS_ORDER']].groupby(['DATE']).sum().reset_index()
df_daily_volume.head()

,DATE,NS_ORDER
0,2017-01-03,4630.0
1,2017-01-04,2690.0
2,2017-01-05,1213.0
3,2017-01-06,3070.0
4,2017-01-07,449.0


In [70]:
#Realizo un gráfico de líneas con plotly
fig = px.line(df_daily_volume, x="DATE", y="NS_ORDER")
fig.update_layout(
    title_text = "Volumen de unidades ordenadas por día",
    title_x = 0.5
)
fig.show()

### Volumen mensual de unidades ordenadas

In [82]:
#Calculo de volumen mensual de unidades ordenadas
df_monthly_volume = df[['DATE', 'NS_ORDER']].copy()
df_monthly_volume['DATE'] = pd.to_datetime(df_monthly_volume['DATE'])
df_monthly_volume['MONTH'] = df_monthly_volume['DATE'].dt.month
df_monthly_summary = df_monthly_volume.groupby('MONTH')['NS_ORDER'].sum().reset_index()


,MONTH,NS_ORDER
0,1,92649.0
1,2,90586.0


In [84]:
# Crear gráfico de barras
fig = px.bar(df_monthly_summary, x='MONTH', y='NS_ORDER', )
fig.update_layout(
    title_text='Unidades ordenadas por mes',
    title_x = 0.5
)
fig.show()

### Análisis ABC por SKUs

In [6]:
#Agrupo las ventas por items y los orden de mayor a menor según la cantidad de ventas
df_sku = df[['ITEM','NS_ORDER']].groupby(['ITEM']).sum().sort_values(by=['NS_ORDER'], ascending = False).reset_index()
df_sku['ITEM'] = df_sku['ITEM'].astype(str)
df_sku.head()

,ITEM,NS_ORDER
0,10098739,27173.0
1,111931,15575.0
2,1041106,13178.0
3,1040765,11980.0
4,110441,9600.0


In [7]:
#Calculo el porcentaje de ventas de cada item y el porcentaje acumulado
df_sku['percentage'] = df_sku['NS_ORDER'] / df_sku['NS_ORDER'].sum()
df_sku['cumulative_percentage'] = df_sku['percentage'].cumsum()
df_sku.head()

,ITEM,NS_ORDER,percentage,cumulative_percentage
0,10098739,27173.0,0.148296,0.148296
1,111931,15575.0,0.085000,0.233296
2,1041106,13178.0,0.071919,0.305215
3,1040765,11980.0,0.065381,0.370595
4,110441,9600.0,0.052392,0.422987


In [8]:
#Categorizar los SKU
def ABC(cum_percentage):
    if cum_percentage >= 0.95:
        category = 'C'
    elif cum_percentage >= 0.8 and cum_percentage < 0.95:
        category = 'B'
    else:
        category = 'A'
    return category

df_sku['category'] = df_sku['cumulative_percentage'].apply(ABC)

In [9]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Agregar gráfico de líneas
fig.add_trace(
    go.Scatter(x=df_sku['ITEM'], y=df_sku['cumulative_percentage'], mode='lines', name='Porcentaje acumulado'),
    secondary_y=False
)

# Agregar gráfico de barras con colores distintos por categoría
colors = px.colors.qualitative.Plotly

categories = df_sku['category'].unique()
for i, category in enumerate(categories):
    df_category = df_sku[df_sku['category'] == category]
    fig.add_trace(
        go.Bar(
            x=df_category['ITEM'],
            y=df_category['percentage'],
            name=f'Porcentaje {category}',
            marker_color=colors[i % len(colors)]  # Cicla los colores si hay más categorías que colores
        ),
        secondary_y=True
    )

# Configurar el rango del eje Porcentaje
fig.update_yaxes(
    range=[0, 1], 
    title_text='Porcentaje',
    secondary_y=True
)

fig.update_layout(
    title_text='Clasificación ABC por SKU',  
    title_x=0.5 
)

fig.show()

Los SKU categoría "A" explican el 80% de las unidades vendidas, los SKU categoría "B" explican el 15% de las unidades vendidas y los SKU categoría "C" explican el 5% de unidades vendidas.

### Análisis ABC por Cliente

In [10]:
#Agrupo las ventas por cliente y los orden de mayor a menor según la cantidad de ventas
df_customer = df[['CUSTOMER_NO','NS_ORDER']].groupby(['CUSTOMER_NO']).sum().sort_values(by=['NS_ORDER'], ascending = False).reset_index()
df_customer['CUSTOMER_NO'] = df_customer['CUSTOMER_NO'].astype(str)
df_customer.head()

,CUSTOMER_NO,NS_ORDER
0,1795849.0,14599.0
1,1255123.0,11629.0
2,1255548.0,11567.0
3,1740542.0,10697.0
4,1254798.0,9584.0


In [11]:
#Calculo el porcentaje de ventas a cada cliente y el porcentaje acumulado
df_customer['percentage'] = df_customer['NS_ORDER'] / df_customer['NS_ORDER'].sum()
df_customer['cumulative_percentage'] = df_customer['percentage'].cumsum()
df_customer.shape

(708, 4)

In [12]:
#Categorizo a los clientes
df_customer['category'] = df_sku['cumulative_percentage'].apply(ABC)

In [13]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Agregar gráfico de líneas
fig.add_trace(
    go.Scatter(x=df_customer['CUSTOMER_NO'], y=df_customer['cumulative_percentage'], mode='lines', name='Porcentaje acumulado'),
    secondary_y=False
)

# Agregar gráfico de barras con colores distintos por categoría
colors = px.colors.qualitative.Plotly

categories = df_customer['category'].unique()
for i, category in enumerate(categories):
    df_category = df_customer[df_customer['category'] == category]
    fig.add_trace(
        go.Bar(
            x=df_category['CUSTOMER_NO'],
            y=df_category['percentage'],
            name=f'Porcentaje {category}',
            marker_color=colors[i % len(colors)] 
        ),
        secondary_y=True
    )

# Configurar el rango del eje Porcentaje
fig.update_yaxes(
    range=[0, 1], 
    title_text='Porcentaje',
    secondary_y=True
)

fig.update_layout(
    title_text='Clasificación ABC por Cliente',  
    title_x=0.5 
)

fig.show()

In [53]:
df_A_customers = df_customer[df_customer['category']== 'A']['CUSTOMER_NO']

De todos los clientes solo 21 acumulan el 80% de las unidades vendidas, estos clientes están categorizados como clientes A.

### Nivel de servicio al cliente
Se analizará el número de ordenes canceladas por linea de despacho, y luego la proporción de ordenes canceladas.

In [15]:
df_canceled = pd.read_csv('canceled.csv')
df_canceled.head()

,ORDER_NO,DATE,LINE,CUSTOMER_NO,ITEM,NC_ORDER,NC_SHIP
0,528703,"Tuesday, January 3, 2017",1,1857566.0,10135139,1,1
1,528705,"Tuesday, January 3, 2017",1,1857566.0,10135140,1,1
2,528706,"Tuesday, January 3, 2017",2,1857566.0,10135138,1,1
3,528707,"Tuesday, January 3, 2017",1,1857566.0,10135132,1,1
4,528708,"Tuesday, January 3, 2017",1,1857566.0,10135359,1,1


In [21]:
df_canceled_line = df_canceled[['LINE','NC_ORDER']].groupby(['LINE']).count().sort_values(by = 'NC_ORDER', ascending= False).reset_index()

#### Cantidad de ordenes canceladas por línea

In [85]:
# Crear gráfico de barras
fig = px.bar(df_canceled_line, x='LINE', y='NC_ORDER', )
fig.update_layout(
    title_text='Ordenes canceladas por linea',
    title_x = 0.5
)
fig.update_xaxes(tickmode='linear')
fig.update_xaxes(tickangle=-90)
fig.show()

#### Proporción de ordenes canceladas por línea

In [37]:
df_acepted_line = df[['LINE','NS_ORDER']].groupby(['LINE']).count().sort_values(by = 'NS_ORDER', ascending= False).reset_index()
df_percentage_acepted = df_acepted_line
df_percentage_acepted['percentage_acepted'] = df_acepted_line['NS_ORDER']/ (df_acepted_line['NS_ORDER'] + df_canceled_line['NC_ORDER'])
df_percentage_acepted['percentage_acepted'] = df_percentage_acepted['percentage_acepted'].fillna(1)
df_percentage_acepted.sort_values(by = 'percentage_acepted', ascending= False, inplace= True)

In [54]:
# Crear gráfico de barras
fig = px.bar(df_percentage_acepted, x='LINE', y='percentage_acepted', )
fig.update_layout(
    title_text='Proporción de ordenes aceptadas por linea',
    title_x = 0.5
)
fig.update_xaxes(tickmode='linear')
fig.update_xaxes(tickangle=-90)
fig.show()

En el gráfico se puede observar que la línea 22, 23, 25 y 30 son las líneas con menor proporción de ordenes completadas.